In [1]:
import pandas as pd
import numpy as np
import re
import scipy.stats
import statsmodels.stats.multitest
import matplotlib.pyplot as plt
from matplotlib import pylab
import seaborn as sns
import cptac

## Unable to get the below code block to bring in the correct tabulated csv information

In [2]:
cluster_df = pd.read_csv(r'C:\Users\joncj\OneDrive\Documents\Research\GitHub\ovarian_somatic_38.maf.3D_Proximity.pairwise.recurrence.l0.r10.clusters', sep = '\t', engine = 'python')

## Importing the csv file information from an excel file for now, not intending to bringing it in this way in the final cookbook

### initial data frame import from excel, contains all of the hotspot cluster information

In [3]:
#This import will bring in the csv from the path on my computer, this is not what the end result will be, but I am trying to bypass the need for excel
#cluster_df = pd.read_excel (r'C:\Users\joncj\OneDrive\Documents\Research\GitHub\WhenMutationsMatter\Jonathan\HS_Cluster.xlsx')

In [4]:
#Iterating through the cluster ID's of the dataframe and compiling a list of all of the cluster ID's that are unique
cluster_list_initial = (cluster_df.Cluster.unique()).tolist()

In [5]:
#Defining the dataframe that will be used to present the information gathered from the cluster dataframe
visualize_df =pd.DataFrame()


cluster_list = list()
for value in cluster_list_initial:
    length = len(cluster_df[cluster_df['Cluster'] == value])
    if length >= 2:
        cluster_list.append(value)

#These two lines are the first step in creating the dataframe that will present the gather information
#Sorting the list of clusters will allow the data to built accurately, and present it in an aesthetic manner
cluster_list.sort()
visualize_df['Cluster_ID'] = cluster_list

#Defining the four different dictionaries that will be built in the following For Loop, each one gather different information
cluster_len_dict = {}
cluster_chrom_dict = {}
cluster_gene_dict = {}
cluster_mut_dict = {}
comparison_dict = {}

#This For Loop iterates through each of the previously identified clusters that are going to be of interest
for value in cluster_list:
    #Each of these lines will create a dictionary to later be used when contructing the dataframe to visualize the desired information
    cluster_len_dict[value] = len(cluster_df[cluster_df['Cluster'] == value])
    cluster_chrom_dict[value] = cluster_df.loc[cluster_df['Cluster'] == value, 'Chromosome'].values[0]
    cluster_gene_dict[value] = cluster_df.loc[cluster_df['Cluster'] == value, 'Gene/Drug'].values[0]
    
    #Within this loop, I am creating a dictionary that uses the cluster ID as the key, and the value as a list of the different mutations that make up the cluster
    mut_list = cluster_df[cluster_df['Cluster'] == value]['Mutation/Gene'].values.tolist()
    cluster_mut_dict[value] = mut_list
    visualize_df.loc[visualize_df['Cluster_ID'] == value, 'Mutation_List_Name'] = 'cluster_mut_dict[' + str(value) + ']'
    
#This is where I construct the different columns of the visualize dataframe with the corresponding parsed information    
visualize_df['Cluster_Size'] = cluster_len_dict.values()
visualize_df['Cluster_Chromosome'] = cluster_chrom_dict.values()
visualize_df['Cluster_Gene/Protein'] = cluster_gene_dict.values()

for key in cluster_gene_dict.keys():
    key_string = str(key)
    if key_string.endswith('0'):
        comparison_dict[cluster_gene_dict[key]] = cluster_mut_dict[key]
    else:
        comparison_dict[str(cluster_gene_dict[key]) + '_' + key_string[-1]] = cluster_mut_dict[key]
    

In [6]:
visualize_df

,Cluster_ID,Mutation_List_Name,Cluster_Size,Cluster_Chromosome,Cluster_Gene/Protein
0,0.0,cluster_mut_dict[0.0],2,chr4,ADH1A
1,9.0,cluster_mut_dict[9.0],3,chr8,CDCA2
2,12.0,cluster_mut_dict[12.0],2,chr2,CPS1
3,25.0,cluster_mut_dict[25.0],2,chr17,EIF4A1
4,29.0,cluster_mut_dict[29.0],2,chr5,FGF18
5,32.0,cluster_mut_dict[32.0],2,chr2,GCA
6,34.0,cluster_mut_dict[34.0],2,chr10,GOT1
7,36.0,cluster_mut_dict[36.0],2,chr19,HNRNPM
8,43.0,cluster_mut_dict[43.0],2,chr11,KDM2A
9,44.0,cluster_mut_dict[44.0],2,chr2,KHK


In [7]:
#visualize_df
#This dataframe will show the following information for each cluster:
    # ID, size, chromosome, gene/protein, as well as a the name of the list that contains the cluster's mutations

## Load the Ovarian dataset

In [8]:
cptac.download(dataset='ovarian')

All files already downloaded and correct.


True

In [9]:
ov = cptac.Ovarian()
proteomics = ov.get_proteomics()
transcriptomics = ov.get_transcriptomics()
clinical = ov.get_clinical()
mutations = ov.get_mutations()

In [12]:
num = 0
visualize_patients_df = pd.DataFrame()
for row in mutations.iterrows():
    info = list(row[1])
    gene = info[0]
    location = info[2]
    if gene in comparison_dict.keys():
        if location in comparison_dict[gene]:
            num += 1

    
    
    
        
        
        

In [13]:
num

41

In [14]:
renal = cptac.RenalCcrcc()
proteomics = renal.get_proteomics()
transcriptomics = renal.get_transcriptomics()
clinical = renal.get_clinical()
mutations = renal.get_mutations()

In [15]:
mutations

,Gene,Mutation,Location
Sample_ID,,,
S001,ACACA,Nonsense_Mutation,p.R662*
S001,ACSL5,Missense_Mutation,p.L699I
S001,AMBRA1,Missense_Mutation,p.Q677L
S001,ARFGEF2,Missense_Mutation,p.E209Q
S001,ASPM,Missense_Mutation,p.R2665P
S001,ATP5B,Frame_Shift_Ins,p.I166Nfs*4
S001,BLVRB,Missense_Mutation,p.S173A
S001,BOD1L1,Missense_Mutation,p.A2303V
S001,C18orf63,Frame_Shift_Del,p.N494Tfs*10
